# Data Engineering Notebook

> TO DO
* Add ADU, Bonus Unit, and Allocation fields to Parcel_History_Attributed
* Clean up unnecessary fields in Parcel_History_Attributed
* Add VHR/Bedrooms to the data
* Add CFA research from Ken to the data
* Check totals year over year
* Add in way to model TAUs as Residential units where City converted Hotels>Apartments

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

#### Terminology 

> Data engineering can consist of ***collection, cleaning, transformation, processing, and automating and monitoring tasks***
* Collection - examples include getting data from a rest service as a
* Cleaning - categorizing 
* Transformation - cateogorizing, standardization, 
* Processing - algorithm, pivot, groupby, merge
* Automating - schedule task, Apache Airflow

> Planning Jargon
* ADU - Accessory Dwelling Unit
* Existing Development Right - refers to residential, commercial, or tourist development currently built in the Lake Tahoe Basin

#### Packages, Maps, and Reference Data

In [2]:
import pandas as pd
import numpy as np
import os
from utils import *
import getpass
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.mapping import show_styles, display_colormaps
from arcgis.gis import GIS
from utils import *
import regex as re

***Pandas Options***

In [3]:
# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 1000)

pd.options.display.float_format = '{:,.2f}'.format
arcpy.env.overwriteOutput = True    
arcpy.env.workspace = 'memory'
# # set workspace
# arcpy.env.workspace = os.path.join(local_path, 'Workspace.gdb')
# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)
# # Set the extent environment using a feature class
# arcpy.env.extent = "Tahoe_OccupancyRate_Zones"


***Map Setup***

In [ ]:
# Set up the GIS object
## portal URL = "https://maps.trpa.org/portal/home/"
## AGOL URL   = "https://www.arcgis.com"
gis = GIS(
    url="https://maps.trpa.org/portal/home/",
    ## enter username above ##
    username= input("Enter username:"),
    ## enter password above ##
    password=getpass.getpass("Enter password:")
)

In [ ]:
# make a map object
map = gis.map("Lake Tahoe", zoomlevel=10)

***Get Reference Data***
* https://www.laketahoeinfo.org/WebServices/List
* https://maps.trpa.org/server/rest/services/

In [ ]:
## LT Info Data
# Verified Development Rights
dfDevRight  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelDevelopmentRightsForAccela/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Deed Restrictions as a DataFrame
dfDeed      = pd.read_json("https://laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# IPES LTinfo as a DataFrame
dfIPES      = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Development Rights Transacted and Banked as a DataFrame
dfDevRights = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# All Parcels as a DataFrame
dfLTParcel  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [ ]:
## TRPA Data 
# Parcel Master as a Spatially Enabled Dataframe from a Feature Service
sdfParcel     = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Parcels/FeatureServer/0")

In [4]:
## TRPA Data 
# Parcel Master as a Spatially Enabled Dataframe from a Feature Service
sdfParcel     = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Parcels/FeatureServer/0")
# TRPA Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfBoundary   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/FeatureServer/4")
# Plan Area Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfPlanArea   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/FeatureServer/0")
# District Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfDistrict   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Zoning/FeatureServer/0")
# Town Center Boundary as a Spatially Enabled Dataframe from a Feature Service
sdfTownCenter = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/FeatureServer/1")

Exception: Could not load the dataset: 'dict' object has no attribute 'lower'

## Permit Data Engineering

#### TRPA Permit Data

***Get Data***
> TRPA permit data is exported from accela nightly then stored in colleciton.sde enterprise geodatabase and published to the trpa server as the web service below

In [ ]:
# web service url
permitTable = "https://maps.trpa.org/server/rest/services/Permit_Records/MapServer/1"
# get permit data as a dataframe
dfTRPAPermit = get_fs_data(permitTable)

In [ ]:
## TRPA Permit Data Engineering
dfTRPAPermit.info()

***Transformation***

In [ ]:
df = dfTRPAPermit

# final fields for all permit dataframes
fields = ['APN', 'Address', 'Jurisdiction', 'Permit_ID', 
          'Permit_Type','Permit_Category', 'Permit_Status',  'Description',
          'Applied_Date', 'Issued_Date', 'PreGrade_Date', 'Finaled_Date'
          ]

# # set fields
column_mapping = {
'Accela_ID' : 'Permit_ID',
'Detailed_Description' : 'Description',
'Record_Status' : 'Permit_Status',
'Accela_CAPType_Name' : 'Permit_Type',
'File_Date' : 'Applied_Date'
}

# rename columns based on dictionary
df = renamecolumns(df, column_mapping, False)

# add missing fields
for field in fields:
    # if field not in dataframe add it
    if field not in df.columns:
        # insert new column
        df[field] = None
# limit to the final fields
df = df[fields]
# add jurisdiction value
df.Jurisdiction = "TRPA"
df.info()


***Processing***

In [ ]:
# print out unique Record_Status values one at a time
for description in dfTRPAPermit.Detailed_Description.unique():
    print(description)

In [ ]:
# print out unique Record_Status values one at a time
for permittype in dfTRPAPermit.Accela_CAPType_Name.unique():
    print(permittype)

In [ ]:
# print out unique Record_Status values one at a time
for status in dfTRPAPermit.Record_Status.unique():
    print(status)

In [ ]:
value_lookup = "resources\Value_Lookups.csv"
trpa_reportingcategory_lookup = import_lookup_dictionary(value_lookup,'key','value','Jurisdiction','TRPA','FieldName','Reporting_Category')
trpa_permittype_lookup        = import_lookup_dictionary(value_lookup,'key','value','Jurisdiction','TRPA','FieldName','Permit_Type')
trpa_permitstatus_lookup      = import_lookup_dictionary(value_lookup,'key','value','Jurisdiction','TRPA','FieldName','Permit_Status')

In [ ]:
# Update fields from lookup dictionaries
df['Reporting_Category'] = df['Reporting_Category'].map(trpa_reportingcategory_lookup)
df['Permit_Type'] = df['Permit_Type'].map(trpa_permittype_lookup)
df['Permit_Status'] = df['Permit_Status'].map(trpa_permitstatus_lookup)

#### City of South Lake Tahoe Permit Data

***Get Data***

In [ ]:
## City of South Lake Tahoe Permit data was sent over by Ryan Malhoski on 4/9/2021
dfCSLTPermit = read_file("data\PermitData_CSLT_040924.csv")

In [ ]:
dfCSLTPermit.info()

***Transformation***

In [ ]:
# drop existing 'Address' field
df = dfCSLTPermit.drop('Address', axis=1)

# final fields for all permit dataframes
fields = ['APN', 'Address', 'Jurisdiction', 
          'Permit_ID', 'Permit_Type','Permit_Status', 'Description',
          'Applied_Date', 'Issued_Date', 'Finaled_Date'
          ]

# # set fields
column_mapping = {
            'Parcel ID': 'APN',
            'Location Address':'Address',
            'Permit Number' : 'Permit_ID',
            'Note Text' : 'Description',
            'Status' : 'Permit_Status',
            'Permit Type' : 'Permit_Type',
            'Permit Issue Date' : 'Applied_Date',
            'Certificate Issue Date': "Finaled_Date"
            }

# rename columns based on dictionary
df = renamecolumns(df, column_mapping,False)

# add missing fields
for field in fields:
    # if field not in dataframe add it
    if field not in df.columns:
        # insert new column
        df[field] = None
# limit to the final fields
df = df[fields]
# add jurisdiction value
df.Jurisdiction = "CSLT"
df.info()

In [ ]:
# APN is a PPNO format in the CSLT data, and also contains EL old naming convetion (-0)
# need to format to xxx-xxx-xxx and filter any odd values (e.g. 500 series)
# get rid of 100's and 500's series, and format to xxx-xxx-xxx, also remove any that start with strings
# strip off trailing spaces
df.APN = df.APN.str.replace(' ', '') 


***Processing***

In [ ]:
# potential values for Permit Type
# 
# get unique permit types
for permittype in dfCSLTPermit["Permit Type"].unique():
    print(permittype)

#### El Dorado County Permit Data
>  there are two files, one for all TRPA files and one for all files in our geographic area, including TRPA files and EDC files. 

***Get Data***

In [ ]:
## El Dorado Permit data representing all files in our geographic area
## exported by Ken Kasman on 4/1/2021 from their Trakit database
dfElDoPermit = read_file("data\PermitData_ElDorado_040124.csv")
dfElDoPermit.info()

***Transformation***

In [ ]:
# drop existing 'Address' field
df = dfElDoPermit

# final fields for all permit dataframes
fields = ['APN', 'Address', 'Jurisdiction', 
          'Permit_ID', 'Permit_Type','Permit_Status','Description',
          'Applied_Date', 'Issued_Date', 'Finaled_Date'
          ]

# # set fields
column_mapping = {
            'SITE_APN' : 'APN',
            'SITE_ADDR':'Address',
            'Permit Number' : 'Permit_ID',
            'DESCRIPTION' : 'Description',
            'STATUS' : 'Permit_Status',
            'PERMITTYPE' : 'Permit_Type',
            'APPLIED' : 'Applied_Date',
            'ISSUED'  : 'Issued_Date',
            'FINALED' : "Finaled_Date"
            }

# rename columns based on dictionary
df = renamecolumns(df, column_mapping, False)

# add missing fields
for field in fields:
    # if field not in dataframe add it
    if field not in df.columns:
        # insert new column
        df[field] = None
# limit to the final fields
df = df[fields]
# add jurisdiction value
df.Jurisdiction = "EL"
df.info()

In [ ]:
for permittype in dfElDoPermit["PERMITTYPE"].unique():
    print(permittype)

In [ ]:
# get lookup dictionary
lookupTable = read_file("resources/lookup_reporting_category.csv")
lookupTable["Reporting Category"].unique()


***Processing***

#### Placer County Permit Data

***Get Data***

In [ ]:
## Placer Permit Data Comes in monthly via email, and gets saved to the folder below.
## The code below will merge all the files in the folder into a single file, return a dataframe, and export to csv

# folder with the CSV files
folder_path = r"F:\Research and Analysis\Local Jurisdiction MOU data collection\Placer MOU Files\Placer"
# List to hold the DataFrames
dfs = []

# Loop through the files in the folder and identify CSV files
for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    # Read the CSV file into a DataFrame and append to the list
    df = pd.read_excel(file_path)
    # Append the DataFrame to the list
    dfs.append(df)
# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)
# Add today's date at the end of the file name _MMDDYY
today = pd.Timestamp.today().strftime("%m%d%y")
# Export the final DataFrame to a CSV file
final_df.to_csv("data\PermitData_Placer_" + today + ".csv", index=False)

In [ ]:
## Placer Permit data explained above. 
dfPlacerPermit =read_file("data\PermitData_Placer_040924.csv")

In [ ]:
dfPlacerPermit.info()

In [ ]:
dfPlacerPermit.head()

***Transformation***
> hyperlink to Placer Accela record can be bulit using SERV_PROD_CODE, B1_PER_ID1, B1_PER_ID2, B1_PER_ID3
* https://permits.placer.ca.gov/CitizenAccess/Cap/CapDetail.aspx?Module=TRPA&TabName=TRPA&capID1=16CAP&capID2=00000&capID3=0036O&agencyCode=PLACERCO

In [ ]:
# create lookup dictionary
lookupTable = read_file("resources/PL_lookup_reporting_category.csv")
lookupTable["Reporting Category"].unique()


***Processing***

#### Merge

In [ ]:
# merege the processed dfs
df = pd.concat([dfTRPA, dfCSLT, dfEL, dfPL], axis=0)

#### Load

In [ ]:
df.to_csv("data\PermitData.csv")

## Cumulative Accounting Data Engineering

#### Existing Development Rights

***Get Data***

In [ ]:
## get 2022 development units
devhistoryURL = "https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2"
# get parcel history for 2023
df23 = get_fs_data_query(devhistoryURL, "Year = 2023")
df22 = get_fs_data_query(devhistoryURL, "Year = 2022")
df21 = get_fs_data_query(devhistoryURL, "Year = 2021")
df20 = get_fs_data_query(devhistoryURL, "Year = 2020")
df19 = get_fs_data_query(devhistoryURL, "Year = 2019")
df18 = get_fs_data_query(devhistoryURL, "Year = 2018")

In [20]:
## get parcel data
# sdfParcel     = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2")
# get spatial data to join to
sdfDistrict   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Planning/MapServer/1")
# sdfPlan       = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/MapServer/0")
# sdfTownCenter = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/MapServer/1")
sdfTCbuffer   = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Planning/MapServer/4")
sdfCSLT       = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/MapServer/2")
# sdfCounty     = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/MapServer/3")
# sdfTRPA       = get_fs_data_spatial("https://maps.trpa.org/server/rest/services/Boundaries/MapServer/4")

Exception: Could not load the dataset: {'code': 500, 'message': 'Error handling service request :0x80004004 -  CreateDynamicMapService failed for service Boundaries.MapServer', 'details': []}

In [6]:
# spatial join to get District
arcpy.SpatialJoin_analysis(sdfParcel, sdfPlan, "Join_PlanArea", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Plan Area, 
arcpy.SpatialJoin_analysis(sdfParcel, sdfDistrict, "Join_District", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Town Center
arcpy.SpatialJoin_analysis(sdfParcel, sdfTownCenter, "Join_TownCenter", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Town Center Buffer
arcpy.SpatialJoin_analysis(sdfParcel, sdfTCbuffer, "Join_TownCenterBuffer", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get CSLT
arcpy.SpatialJoin_analysis(sdfParcel, sdfCSLT, "Join_CSLT", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get County
arcpy.SpatialJoin_analysis(sdfParcel, sdfCounty, "Join_County", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get TRPA Boundary
arcpy.SpatialJoin_analysis(sdfParcel, sdfTRPA, "Join_TRPA", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")


NameError: name 'sdfPlan' is not defined

In [ ]:
# get results of spatial joins as spatial dataframes
sdf_parcel_plan = pd.DataFrame.spatial.from_featureclass("Join_PlanArea", sr=sr)  
sdf_parcel_zone = pd.DataFrame.spatial.from_featureclass("Join_Zone", sr=sr)


# map dictionary to sdf_units dataframe to fill in TAZ and Block Group fields
sdfParcel['PLAN_AREA']     = sdfParcel.APN.map(dict(zip(sdf_parcel_plan.APN, sdf_parcel_plan.PLAN_NAME)))
sdfParcel['BLOCK_GROUP']   = sdfParcel.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
sdfParcel['OCCUPANCY_ZONE']= sdfParcel.APN.map(dict(zip(sdf_units_occ.APN,   sdf_units_occ.OccupancyRate_ZoneID)))

In [ ]:
# create spatial dataframe from spatial join feature class
sdf_PlanJoin   = pd.DataFrame.spatial.from_featureclass("Join_PlanArea", sr=sr)
sdf_ZoneJoin   = pd.DataFrame.spatial.from_featureclass("Join_Zone", sr=sr) 

In [ ]:
# map dictionary to sdf_units dataframe to fill in TAZ and Block Group fields
df['PLAN_NAME_NEW'] = sdfParcel.APN.map(dict(zip(sdf_PlanJoin.APN, sdf_PlanJoin.NAME)))
df['PLAN_ID_NEW']   = sdfParcel.APN.map(dict(zip(sdf_PlanJoin.APN, sdf_PlanJoin.ID)))
df['ZONE_NAME']     = sdfParcel.APN.map(dict(zip(sdf_ZoneJoin.APN, sdf_ZoneJoin.NAME)))
df['ZONE_ID']       = sdfParcel.APN.map(dict(zip(sdf_ZoneJoin.APN, sdf_ZoneJoin.ID)))


In [ ]:
sdf_PlanJoin

In [ ]:
# get the plan name and id
df.groupby(['PLAN_ID', 'YEAR']).agg({'Residential_Units':'sum'}).reset_index()

In [ ]:
df.pivot_table(index='APN', columns='YEAR', values=['Residential_Units','CommercialFloorArea_SqFt', 'TouristAccommodation_Units'], aggfunc='sum').reset_index()


In [ ]:
## get 2022 development units
devhistoryURL = "https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2"
# get parcel history for 2023
df = get_fs_data_query(devhistoryURL, "Year = 2023")

In [ ]:
# # groub by EXISTING_LANDUSE and aggregate Residential Units
# df.groupby("EXISTING_LANDUSE")["Residential_Units"].sum()
# flatten out as a dataframe
df.groupby("EXISTING_LANDUSE")["Residential_Units"].sum().reset_index()
# add a total field


In [ ]:
# display all rows
pd.set_option('display.max_rows', None)

# group by PLAN_NAME and sum Residential_Units
df1 = df.groupby('PLAN_ID').agg({'Residential_Units':'sum', 'TouristAccommodation_Units':'sum','CommercialFloorArea_SqFt':'sum'}).reset_index()

# print
df1.sort_values('Residential_Units', ascending=False)

# add total row
df1.loc['Total'] = df1.sum(numeric_only=True, axis=0)

In [ ]:
# # export to CSV
# df = parcel_history

# columns to keep
columns_to_keep = ['APN', 'Residential_Units', 'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt', 'YEAR',
                    'JURISDICTION', 'COUNTY', 
                    # 'ADU', 'RBU', 'Allocation','Deed_Restricted_Units',
                    'OWNERSHIP_TYPE','EXISTING_LANDUSE',
                    # 'WITHIN_TRPA_BNDY'
                    'PARCEL_ACRES', 'PARCEL_SQFT']

# add integer columns for RBU, ADU, Allocation, and Deed Restricted Units
df['ADU'] = 0
df['RBU'] = 0
df['Allocation'] = 0
df['Deed_Restricted_Units'] = 0

# keep only the columns in the list
df = df[columns_to_keep]
# # filter to 2023
# df = df[df.YEAR == 2023]

# export to csv with date stamp in name
today = pd.Timestamp.today().strftime("%m%d%y")
df.to_csv("data\DevelopmentHistory_2023_" + today + ".csv")

In [ ]:
## get 2022 development units
devhistoryURL = "https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2"
parcel_history = get_fs_data_spatial(devhistoryURL)

# get unit table as pandas dataframe
unitsTable = pd.read_csv("data/CumulativeAccounting_2012to2023_Updated.csv", low_memory=False)
# get rid of columns after YEAR
unitsTable.drop(unitsTable.columns[unitsTable.columns.get_loc("YEAR")+1:], axis=1,inplace=True)
# set cfa to numeric
unitsTable['CommercialFloorArea_SqFt'] = pd.to_numeric(unitsTable['CommercialFloorArea_SqFt'], errors='coerce').fillna(0)  


In [ ]:
# global variables
years = [2012, 2018, 2019, 2020, 2021, 2022, 2023]
version = "_v6_"

# merge parcel history and units table by year and 
# export to feature class
def merge_and_export(parcel_history, unitsTable, years):
    for year in years:
        print(year)
        # filter parcel_history by year
        parcel_history_year = parcel_history.loc[parcel_history['YEAR'] == year]
        # filter unitsTable by year
        unitsTable_year = unitsTable.loc[unitsTable['YEAR'] == year]
        # merge parcel_history_year and unitsTable_year
        df = pd.merge(parcel_history_year, unitsTable_year, on='APN', how='left', indicator=True)
        # make sure field types are numeric for Residential_Unit, TouristAccommodation_Units, and CommercialFloorArea_SqFt fields
        df['Residential_Units']          = pd.to_numeric(df['Residential_Units_y'], errors='coerce')
        df['TouristAccommodation_Units'] = pd.to_numeric(df['TouristAccommodation_Units_y'], errors='coerce')
        df['CommercialFloorArea_SqFt']   = pd.to_numeric(df['CommercialFloorArea_SqFt_y'], errors='coerce')
        # if NaN in Residential_Units, set to 0
        df['Residential_Units'] = df['Residential_Units'].fillna(0)
        # if NaN in TouristAccommodation_Units, set to 0
        df['TouristAccommodation_Units'] = df['TouristAccommodation_Units'].fillna(0)
        # if NaN in CommercialFloorArea_SqFt, set to 0
        df['CommercialFloorArea_SqFt'] = df['CommercialFloorArea_SqFt'].fillna(0)
        # change YEAR_y to YEAR
        df['YEAR'] = df['YEAR_y']
        # Sanitize column names
        df.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in df.columns]
        # set output feature class name
        yearstr = str(year)
        outfc = f"Parcel_History_Attributed{version}{yearstr}"    
        # export updated parcel history to feature class filtered by year
        df.spatial.to_featureclass(location=os.path.join("C:/GIS/Scratch.gdb", outfc), overwrite=True, sanitize_columns=False)

# identify parcels that did not join from the merge
def get_unjoined(parcel_history, unitsTable, years):
    for year in years:
        # Filter parcel_history for the current year
        parcel_history_filtered = parcel_history.loc[parcel_history['YEAR'] == year]
        
        # Merge with unitsTable for the same year
        units_by_year = unitsTable.loc[unitsTable.YEAR == year]
        merged_data = units_by_year.merge(parcel_history_filtered, on='APN', how='outer', indicator=True)
        
        # Print year and merge value counts
        print(year)
        print(merged_data._merge.value_counts())
        
        # Data manipulations
        merged_data = merged_data.rename(columns={'YEAR_x': 'YEAR'})
        merged_data = merged_data.loc[merged_data._merge != 'both']
        merged_data.info()
        merged_data = merged_data[['APN', 'YEAR', '_merge', 'CommercialFloorArea_SqFt_x', 'Residential_Units_x', 'TouristAccommodation_Units_x',
                                   'CommercialFloorArea_SqFt_y', 'Residential_Units_y', 'TouristAccommodation_Units_y']]
        merged_data.info()
        # Save to CSV
        merged_data.to_csv(f"data\\Parcel_History_Attributed_APN_Merge{version, year}.csv", index=False)

# check for duplicates in parcel_history
def check_duplicates(parcel_history, unitsTable, years):
    for year in years:
        print(year)
        # make a list of duplicate APNs
        duplicateAPNs = parcel_history.loc[parcel_history['YEAR'] == year].APN[parcel_history.loc[parcel_history['YEAR'] == year].APN.duplicated()].tolist()
        # print out duplicate rows
        print(duplicateAPNs)
        # save 2021 duplicates to csv
        if year == 2018:
            parcel_history.loc[parcel_history['YEAR'] == year].loc[parcel_history.loc[parcel_history['YEAR'] == year].APN.duplicated()].to_csv("data\Parcel_History_Duplicates_2018.csv")

        # make a list of duplicate APNs in unitsTable
        duplicateAPNsCA = unitsTable.loc[unitsTable['YEAR'] == year].APN[unitsTable.loc[unitsTable['YEAR'] == year].APN.duplicated()].tolist()
        # print out duplicate rows
        print(duplicateAPNsCA)

# compare total Residnetial Units, Commercial Floor Area, and Tourist Accommodation Units by year, bewtween parcel_history and unitsTable
def compare_totals(parcel_history, unitsTable, years):
    for year in years:
        # filter parcel_history by year
        parcel_history_year = parcel_history.loc[parcel_history['YEAR'] == year]
        # filter unitsTable by year
        unitsTable_year = unitsTable.loc[unitsTable['YEAR'] == year]
        # # remove any commas from CommercialFloorArea_SqFt in unitsTable_year using .loc
        # unitsTable_year.loc[:, 'CommercialFloorArea_SqFt'] = unitsTable_year['CommercialFloorArea_SqFt'].str.replace(',', '').astype(float)

        # get sum of Residential Units in parcel_history
        resTotal = parcel_history_year['Residential_Units'].sum()
        cfaTotal = parcel_history_year['CommercialFloorArea_SqFt'].sum()
        tauTotal = parcel_history_year['TouristAccommodation_Units'].sum()

        # get sum of Residential Units in unitsTable
        resTotalCA = unitsTable_year['Residential_Units'].sum()
        cfaTotalCA = unitsTable_year['CommercialFloorArea_SqFt'].sum()
        tauTotalCA = unitsTable_year['TouristAccommodation_Units'].sum()

        # print totals
        print(year)
        print('Residential Units in Parcel_History \n' + str(resTotal))
        print('Residential Units in updated table \n'+ str(resTotalCA))
        print('Commercial Floor Area in Parcel_History \n'+ str(cfaTotal))
        print('Commercial Floor Area in updated table \n'+ str(cfaTotalCA))
        print('Tourist Accommodation Units in Parcel_History \n'+ str(tauTotal))
        print('Tourist Accommodation Units in updated table \n'+ str(tauTotalCA))

# identify rows where the Residential Units, Commercial Floor Area, and Tourist Accommodation Units are different between parcel_history and unitsTable
def find_different_rows(parcel_history, unitsTable, years):
    for year in years:
        print(year)
        # filter parcel_history by year
        parcel_history_year = parcel_history.loc[parcel_history['YEAR'] == year]
        # filter unitsTable by year
        unitsTable_year = unitsTable.loc[unitsTable['YEAR'] == year]
        # # remove any commas from CommercialFloorArea_SqFt in unitsTable_year using .loc
        # unitsTable_year.loc[:, 'CommercialFloorArea_SqFt'] = unitsTable_year['CommercialFloorArea_SqFt'].str.replace(',', '').astype(float)
        # merge parcel_history_year and unitsTable_year
        df = pd.merge(parcel_history_year, unitsTable_year, right_on='APN', left_on='APN', how='outer', indicator=True)
        # drop columns that are not needed
        df = df[['APN', 'YEAR_x','YEAR_y', 'Residential_Units_x', 'CommercialFloorArea_SqFt_x', 'TouristAccommodation_Units_x', 'Residential_Units_y', 'CommercialFloorArea_SqFt_y', 'TouristAccommodation_Units_y']]
        # get fields where the Residential Units, Commercial Floor Area, and Tourist Accommodation Units do not match
        df = df.loc[(df['Residential_Units_x'] != df['Residential_Units_y']) | (df['CommercialFloorArea_SqFt_x'] != df['CommercialFloorArea_SqFt_y']) | (df['TouristAccommodation_Units_x'] != df['TouristAccommodation_Units_y'])]
        # print out the rows
        print(df)

In [ ]:
# run the merge functions to export feature classes and get unjoined data as csv
# merge_and_export(parcel_history, unitsTable, years)
get_unjoined(parcel_history, unitsTable, years)
check_duplicates(parcel_history, unitsTable, years)
compare_totals(parcel_history, unitsTable, years)
find_different_rows(parcel_history, unitsTable, years)

In [ ]:
# analyze the changes in parcel history by year
years = [2012, 2018, 2019, 2020, 2021, 2022, 2023]
df = sdfUnits
for year in years:
    print(year)
    # filter parcel_history by year
    parcel_history_year = df.loc[df['YEAR'] == year]
    # get sum of Residential Units in parcel_history
    resTotal = parcel_history_year['Residential_Units'].sum()
    cfaTotal = parcel_history_year['CommercialFloorArea_SqFt'].sum()
    tauTotal = parcel_history_year['TouristAccommodation_Units'].sum()
    # print totals
    print('Residential Units in Parcel_History \n' + str(resTotal))
    print('Commercial Floor Area in Parcel_History \n'+ str(cfaTotal))
    print('Tourist Accommodation Units in Parcel_History \n'+ str(tauTotal))
    # print out changes in units by APN
# firnd all the rows where duplicate APNs change units between years
for year in years:
    print(year)
    # make a list of duplicate APNs as sets of APNs
    duplicateAPNs = df.loc[df['YEAR'] == year].APN[df.loc[df['YEAR'] == year].APN.duplicated()].tolist()
    # loop through the duplicate APNs
    for apn in duplicateAPNs:
        # get the rows for the APN
        df = df.loc[df['APN'] == apn]
        # get the rows for the APN by year
        df = df.loc[df['YEAR'] == year]
    

In [ ]:
# get total residential units by year
def get_totals(parcels, years):
    # total
    total = pd.DataFrame(columns=['Year', 'Residential_Units'])
    for year in years:
        # filter parcel_history by year
        parcel_history_year = parcels.loc[parcels['YEAR'] == year]
        # get sum of Residential Units in parcel_history
        resTotal = parcel_history_year['Residential_Units'].sum()

        # add new row using concat
        total = pd.concat([total, pd.DataFrame({'Year': [year], 'Residential_Units': [resTotal]})])
    return total

# get total residential units by year
total = get_totals(parcel_history, years)
# calculate percentage change in residential units year over year
total['Percent_Change'] = (total['Residential_Units'].pct_change())*100
# create a new column for the difference in residential units year over year
total['Difference'] = total['Residential_Units'].diff()

total
# export to csv
total.to_csv('total_residential_units_by_year.csv', index=False)


***Transformation***

***Proecssing***

#### Deed Restrictions
> Deed restricted unit research needs to be merged with LTinfo housing deed restricitons and parcel unit data from 2022

***Get Data***

In [ ]:
dfDeedUnits  = read_excel("data\Housing_Deed_Restrcitions.xlsx", 0)
dfDeedLTinfo = pd.read_json("https://laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [ ]:
dfDeedUnits.to_csv("data\DeedRestricted_HousingUnits.csv", index=False)

In [ ]:
dfDeedUnits.Units.sum()

In [ ]:
dfDeedLTinfo.info()

In [ ]:
# get unique values for deed restrcition type
dfDeedLTinfo.DeedRestrictionType.unique()

# filter to Affordable, Achievable, and Moderate
dfDeedLTinfo = dfDeedLTinfo[dfDeedLTinfo.DeedRestrictionType.isin(['Affordable Housing', 'Moderate Income Housing', 'Achievable Housing'])]  

# count of total records
dfDeedLTinfo.shape[0]

In [ ]:
parcelUnits22.info()

In [ ]:
dfDeedUnitsMerge = dfDeedUnits.merge(dfDeedLTinfo, on='APN', how='outer', indicator=True)

In [ ]:
dfDeedUnitsMerge._merge.value_counts()

In [ ]:
dfDeedLTinfo[dfDeedLTinfo.duplicated(subset=['APN','DeedRestrictionType'], keep=False)].sort_values('APN').to_csv("HousingDeedRestrictions_LTinfo_Duplicates.csv")

In [ ]:
# identify duplicates unique by APN and 
dfDeedUnits[dfDeedUnits.duplicated(subset=['APN', 'Deed_Restriction_Type','Units'], keep=False)]

In [ ]:
# identify duplicates
dfDeedUnitsMerge[dfDeedUnitsMerge.duplicated(subset=['APN'], keep=False)].sort_values(by='APN')

In [ ]:
dfDeedUnitsMerge.to_csv("HousingDeedRestrictions_All.csv")

In [ ]:
# merge the deed restricted units with the parcel units
dfDeedUnits_ParcelUnits  = dfDeedUnits.merge(parcelUnits22, on='APN', how='left')
# merge the deed restricted units with the parcel units
dfDeedLTinfo_ParcelUnits = dfDeedLTinfo.merge(parcelUnits22, left_on='APN', right_on='APN', how='left')


In [ ]:
dfDeedLTinfo_ParcelUnits.info()

In [ ]:
dfDeedLTinfo_ParcelUnits.Residential_Units.sum()

#### ADU Tracking
> ADU permit tracking from TRPA and othe Jurisdictions. There is a need to establish a system of record for this information (LT Info). This is similar to the Residential Bonus Unit data and there’s crossover on some of these, where a bonus unit was used to create an ADU, but you can have an ADU without requiring a bonus unit, and you can use a bonus unit without it being an ADU… 

***Get Data***

In [ ]:
dfADU = read_excel("data\ADU Tracking.xlsx", 0)

In [ ]:
dfADU

#### Allocations
> This file includes all of the allocations that have been tracked in LT Info, and adds in whether the subject parcel has been issued a BMP/SCC certificate and/or whether Air Quality/Mobility Mitigation fees (for added VMT) or Water Quality Mitigation fees (for added coverage) have been paid. 

In [ ]:
allocations = read_excel("data\Allocation_Tracking.xlsx", 0)

#### Transactions with Inactive APNs

In [ ]:
inactiveParcels = read_file("data\Transactions_InactiveParcels.csv")

## QA Process

> Process to compare against assessor parcel data signifying development


In [ ]:
# create parcels feature class of missing parcels for Residential Units

# get parcel master
parcelURL = "https://maps.trpa.org/server/rest/services/Parcels/MapServer/0"
vhrURL    = "https://maps.trpa.org/server/rest/services/VHR/MapServer/0"
# get parcel and VHR data as spatial dataframes
sdfParcel = get_fs_data_spatial(parcelURL)
# sdfVHR    = get_fs_data_spatial(vhrURL)

# # keep only the columns needed
# sdfParcel = sdfParcel[['APN','EXISTING_LANDUSE','YEAR_BUILT','BEDROOMS','UNITS','SHAPE']]
# sdfVHR    = sdfVHR[['APN','SHAPE']]

In [ ]:
# Gets feature service data as spatially enabled dataframe
def get_fs_data_spatial(service_url):
    feature_layer = FeatureLayer(service_url)
    df = feature_layer.query().sdf
    return df

In [ ]:
# merge the parcel and VHR data
sdf = pd.merge(sdfParcel, sdfVHR, on='APN', how='left', indicator=True)
# merge the 2023 parcelhistory and 
parcelDev2023 = parcel_history.loc[parcel_history['YEAR'] == 2023]
sdf = pd.merge(sdf, parcelDev2023, on='APN', how='left', indicator=True)
sdf.info()
# # keep fields needed for QA
# sdf = sdf[['APN','EXISTING_LANDUSE','YEAR_BUILT','BEDROOMS','UNITS','WITHIN_TRPABNDY','_merge','SHAPE']]
# # export to feature class
# sdf.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace, 'Parcel_Review'), overwrite=True, sanitize_columns=False)